In [ ]:
import datetime

import cartopy.crs as ccrs
import geoviews as gv
import holoviews as hv
import panel as pn
import param
from holoviews import streams

pn.extension()
gv.extension("bokeh")

In [ ]:
class SpatialExtent(param.Parameterized):
    """A bounding box in decimal degrees"""

    longitude_min: float = param.Number(default=0, bounds=(0, 360), allow_None=False, constant=True)  # type: ignore
    longitude_max: float = param.Number(default=0, bounds=(0, 360), allow_None=False, constant=True)  # type: ignore
    latitude_min: float = param.Number(default=0, bounds=(-90, 90), allow_None=False, constant=True)  # type: ignore
    latitude_max: float = param.Number(default=0, bounds=(-90, 90), allow_None=False, constant=True)  # type: ignore

    @property
    def center(self) -> tuple[float, float]:
        """Returns the midpoint of the bounding box (longitude, latitude)"""
        return ((self.longitude_min + self.longitude_max) / 2, (self.latitude_min + self.latitude_max) / 2)

    @property
    def polygon(self) -> gv.Polygons:
        """Returns a Polygons object representing the spatial bounding box"""
        # right-hand
        polygons = gv.Polygons(
            [
                [
                    (self.longitude_min, self.latitude_min),
                    (self.longitude_min, self.latitude_max),
                    (self.longitude_max, self.latitude_max),
                    (self.longitude_max, self.latitude_min),
                    (self.longitude_min, self.latitude_min),
                ]
            ],
            kdims=["Longitude", "Latitude"],
        )
        polygons.opts(fill_alpha=0, line_color="blue", line_width=2)
        return polygons


In [ ]:
class TemporalExtent(param.Parameterized):
    t_min: datetime.datetime = param.Date(default=datetime.datetime(2000, 1, 1), allow_None=False, constant=True)  # type: ignore
    t_max: datetime.datetime = param.Date(default=datetime.datetime(2024, 12, 31), allow_None=False, constant=True)  # type: ignore

In [ ]:
class Extent(param.Parameterized):
    """
    Mirrors STAC catalog extent.
    """

    spatial: SpatialExtent = param.ClassSelector(
        class_=SpatialExtent, default=SpatialExtent(), allow_None=False, constant=True
    )  # type: ignore
    temporal: TemporalExtent = param.ClassSelector(
        class_=TemporalExtent, default=TemporalExtent(), allow_None=False, constant=True
    )  # type: ignore

    @staticmethod
    def from_json(json: dict) -> "Extent":
        """
        Create an Extent object from STAC JSON.

        https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md#extents

        ```
        {
            "spatial": {
                "bbox": [
                    [
                        19.4342728416868,
                        64.0867100842452,
                        19.702569859525383,
                        64.2372650939366
                    ]
                ]
            },
            "temporal": {
                "interval": [
                    [
                        "2013-06-02T00:00:00Z",
                        "2023-06-30T00:00:00Z"
                    ]
                ]
            }
        }
        ```
        """
        bbox = json["spatial"]["bbox"][0]
        if len(bbox) != 4:
            raise ValueError("Invalid bounding box length, expected 4 values.")
        spatial = SpatialExtent(
            longitude_min=bbox[0], latitude_min=bbox[1], longitude_max=bbox[2], latitude_max=bbox[3]
        )
        interval = json["temporal"]["interval"][0]
        t_min = datetime.datetime.fromisoformat(interval[0])
        t_max = datetime.datetime.fromisoformat(interval[1])
        temporal = TemporalExtent(t_min=t_min, t_max=t_max)
        return Extent(spatial=spatial, temporal=temporal)

In [ ]:
class XYT(pn.viewable.Viewer):
    """
    A utility Parameterized class that contains

    - a point of interest (lat, lon)
    - a date of interest

    along with a Pane for displaying and editing these values.
    """

    extent: Extent = param.ClassSelector(class_=Extent, default=Extent(), allow_None=False, constant=True)  # type: ignore

    latitude: float = param.Number(default=0.0, bounds=(-90, 90), allow_None=False)  # type: ignore
    longitude: float = param.Number(default=0.0, bounds=(-180, 180), allow_None=False)  # type: ignore
    date: datetime.datetime = param.Date(default=datetime.datetime(2000, 1, 1), allow_None=False)  # type: ignore

    def __init__(self, **params):
        super().__init__(**params)

        # set extents on lat, lon, date
        self.param.latitude.bounds = (
            self.extent.spatial.latitude_min,
            self.extent.spatial.latitude_max,
        )
        self.param.longitude.bounds = (
            self.extent.spatial.longitude_min,
            self.extent.spatial.longitude_max,
        )
        self.param.date.bounds = (
            self.extent.temporal.t_min,
            self.extent.temporal.t_max,
        )

        # reassign values to trigger validation
        (x, y) = self.extent.spatial.center
        if "latitude" in params:
            self.latitude = params["latitude"]
        else:
            # set to middle of the extent
            self.latitude = y
        if "longitude" in params:
            self.longitude = params["longitude"]
        else:
            # set to middle of the extent
            self.longitude = x
        if "date" in params:
            self.date = params["date"]
        else:
            # set to t_min
            self.date = self.extent.temporal.t_min

    def map_view(self):
        """
        GeoViews plot in Google web mercator projection with a basemap layer.
        Shows the bounding box of the extent, and the (dynamic) point of interest.
        Taps on the map update the point of interest.
        """
        basemap = gv.tile_sources.OSM
        bbox = self.extent.spatial.polygon

        def point(x, y) -> gv.Points:
            """Create a point at the given x, y coordinates."""
            points = gv.Points([(x, y)], kdims=["longitude", "latitude"])
            points.opts(color="red", size=10)
            return points

        point_dmap = hv.DynamicMap(point, streams={"x": self.param.longitude, "y": self.param.latitude})

        tap = streams.Tap(source=point_dmap)

        def on_click(x, y):
            """
            Args:
                x: The x-position of a tap or click in data coordinates.
                y: The y-position of a tap or click in data coordinates.
            """
            try:
                # "transactional" update of both parameters
                self.param.update(longitude=x, latitude=y)
            except ValueError:
                # Ignore clicks outside the bounds
                pass

        tap.add_subscriber(on_click)

        plot = basemap * bbox * point_dmap
        plot.opts(projection=ccrs.GOOGLE_MERCATOR)

        return plot

    def __panel__(self) -> pn.viewable.Viewable:
        return pn.Row(
            pn.Param(
                self,
                parameters=["latitude", "longitude", "date"],
                show_name=False,
                widgets={
                    "latitude": pn.widgets.FloatInput,
                    "longitude": pn.widgets.FloatInput,
                    "date": {
                        "type": pn.widgets.DatetimeSlider,
                        "start": self.extent.temporal.t_min,
                        "end": self.extent.temporal.t_max,
                        "step": 60 * 60,  # 1 hour step
                        "throttled": True,
                    },
                },
            ),
            pn.pane.HoloViews(self.map_view()),
        )

In [ ]:
extent = Extent(
    spatial=SpatialExtent(longitude_min=5, longitude_max=10, latitude_min=5, latitude_max=10),
)

In [ ]:
obj = XYT(extent=extent)

In [ ]:
repr(obj)

In [ ]:
obj